# Pretrained Model을 이용한 RAG

In [ ]:
%%capture
!pip install  git+https://github.com/huggingface/peft.git
!pip install bitsandbytes
!pip install accelerate==0.21.0
!pip install datasets

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Pytorch Import
import torch
import torch.nn as nn
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

## Transforemr Import
import transformers
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, BitsAndBytesConfig
from transformers import AutoTokenizer, AdamW, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, AutoModelForCausalLM

## Accerate
from accelerate import Accelerator

# Tqdm
from tqdm.auto import tqdm, trange

# HuggingFace peft
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType, prepare_model_for_kbit_training
from peft import PeftModel, PeftConfig

# Dataset
import datasets
from datasets import Dataset, DatasetDict, load_dataset

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os

base_path = "/content/drive/MyDrive/contest/dacon_hansol_llm"
data_path = os.path.join(base_path, "data")
model_save = os.path.join(base_path,'model/KoAlpaca/')
# sub_path = base_path + 'sub/KoAlaca/'

# BASE 모델 지정

In [ ]:
model_id = "beomi/polyglot-ko-12.8b-safetensors"  # safetensors 컨버팅된 레포

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/52.5k [00:00<?, ?B/s]

model-00001-of-00028.safetensors:   0%|          | 0.00/946M [00:00<?, ?B/s]

model-00002-of-00028.safetensors:   0%|          | 0.00/843M [00:00<?, ?B/s]

model-00003-of-00028.safetensors:   0%|          | 0.00/843M [00:00<?, ?B/s]

...

model-00026-of-00028.safetensors:   0%|          | 0.00/1.00G [00:00<?, ?B/s]

model-00027-of-00028.safetensors:   0%|          | 0.00/896M [00:00<?, ?B/s]

model-00028-of-00028.safetensors:   0%|          | 0.00/518M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/100 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# BASE 모델에 Pretrained QLoRa 불러오기

In [ ]:
# Load the Lora model
model = PeftModel.from_pretrained(model = model, model_id = model_save + f'output_peft_dir', device_map={"":0})

In [ ]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30080, 5120)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-39): 40 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXAttention(
              (rotary_emb): GPTNeoXRotaryEmbedding()
              (query_key_value): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=5120, out_features=15360, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                

# RAG 파이프라인

In [ ]:
%%capture
!pip install faiss-gpu
!pip install sentence_transformers
!pip install langchain

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import glob
import textwrap
import time

import langchain

# loaders
# from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

# splits
from langchain.text_splitter import RecursiveCharacterTextSplitter

# prompts
from langchain import PromptTemplate, LLMChain

# vector stores
from langchain.vectorstores import FAISS

# models
from langchain.llms import HuggingFacePipeline
# from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceEmbeddings # HuggingFaceInstructEmbeddings

# retrievers
from langchain.chains import RetrievalQA

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# from peft import (
#     LoraConfig,
#     PeftConfig,
#     get_peft_model,
#     prepare_model_for_kbit_training, # 4bir Qlora
# )
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

print('LangChain:', langchain.__version__)

LangChain: 0.1.8


In [ ]:
class CFG:
    # LLMs
    # model_name = "beomi/llama-2-ko-7b" # "psymon/KoLlama2-7b" -> 4bit or 8bit model 없음
    temperature = 0,
    top_p = 0.95,
    repetition_penalty = 1.15

    # splitting
    # split_chunk_size = 800
    # split_overlap = 0

    # embeddings
    embeddings_model_repo = 'distiluse-base-multilingual-cased-v1'
    Embeddings_path = "/content/drive/MyDrive/contest/dacon_hansol_llm/vectordb"
    # similar passages
    k = 3

    max_len = 512

In [ ]:
pipe = pipeline(
    task = "text-generation",
    model = model,
    tokenizer = tokenizer,
    pad_token_id = tokenizer.eos_token_id,
    # max_length = CFG.max_len,
    temperature = CFG.temperature,
    top_p = CFG.top_p,
    repetition_penalty = CFG.repetition_penalty,
    max_new_tokens=512
    # device=0 # 양자화 버전으로 다운 받아서 device로 이동 못한다는 에러 뜸
)

# pipe.save_pretrained(root_path+'/RAG')

llm = HuggingFacePipeline(pipeline = pipe)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonF

# 데이터 가져오기 & 전처리

In [ ]:
# train = pd.read_csv(data_path + "train_processed.csv")

# train_1 = pd.melt(train, id_vars=['질문_1'], value_vars=['답변_1', '답변_2', '답변_3', '답변_4', '답변_5'], var_name='답변번호', value_name='답변')
# train_1.columns = ['instruction','answer_no','output']
# train_2 = pd.melt(train, id_vars=['질문_2'], value_vars=['답변_1', '답변_2', '답변_3', '답변_4', '답변_5'], var_name='답변번호', value_name='답변')
# train_2.columns = ['instruction','answer_no','output']

# train = pd.concat([train_1,train_2]).sort_values(by=['instruction','output']).reset_index(drop=True)
# train = train[['instruction','output']]

In [ ]:
train_data = pd.read_csv(os.path.join(base_path, "data/train.csv"))
test_data = pd.read_csv(os.path.join(base_path, "data/test.csv"))

In [ ]:
from itertools import product

train_data_list = []

for q,a in list(product([f"질문_{x}" for x in range(1,3)],[f"답변_{x}" for x in range(1,6)])):
    for i in range(len(train_data)):
        train_data_list.append(
            "질문: "+ train_data.at[i,q] + " 답변 : " + train_data.at[i,a]
        )

In [ ]:
model_kwargs = {'device':'cuda'}

encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name=CFG.embeddings_model_repo,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [ ]:
### create embeddings and DB
vectordb = FAISS.from_texts(
    texts = train_data_list,
    embedding = embeddings
)

In [ ]:
retriever = vectordb.as_retriever(search_kwargs = {"k": CFG.k, "search_type" : "similarity"})

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


prompt_template = """
마지막에 질문에 답하려면 다음과 같은 맥락을 사용합니다.
{context}

질문: {question}
답변:"""


PROMPT = PromptTemplate(
    template = prompt_template,
    input_variables = ["context", "question"]
)

In [ ]:
retriever = vectordb.as_retriever(search_kwargs = {"k": CFG.k, "search_type" : "similarity"})

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    verbose=True,
    chain_type_kwargs={"prompt": PROMPT},
)

# 응답테스트

# a. 관련 문장 가져오기

In [ ]:
query = '방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요?'
vectordb.max_marginal_relevance_search(query, k = CFG.k)

[Document(page_content='질문: 어떤 표면에 사용할 페인트가 있는지 알고 싶은데, 어떤 종류가 있나요? 답변 : 페인트의 종류로 수성페인트, 유성페인트, 천연페인트가 있습니다.'),
 Document(page_content='질문: 페인트에 포함된 유해물질은 어떤 것들이 있나요? 답변 : 페인트에는 주로 벤젠, 톨루엔, 자일렌과 같은 유해물질이 함유되어 있습니다. 이러한 물질들은 페인트를 사용할 때 호흡기나 피부를 통해 체내로 흡수되어 건강에 해를 끼칠 수 있습니다. 특히, 페인트에 함유된 VOCs(휘발성유기화합물)는 발암 물질로 알려져 있어 새집증후군과 같은 문제를 야기할 수 있습니다. 이에 따라 페인트 작업 시에는 적절한 환기 시설을 설치하고, 마스크와 같은 개인보호구를 착용하는 것이 매우 중요합니다. 또한, 환경부는 휘발성유기화합물의 위험에 대비해 실내에서 유성페인트 사용을 금지하고, 수성페인트를 사용하는 것을 권장하고 있습니다.'),
 Document(page_content='질문: 방청 페인트의 종류는 뭐가 있어? 답변 : 방청 페인트의 종류로는 광명단 페인트, 방청산화철 페인트, 알루미늄 페인트, 역청질 페인트, 워시 프라이머, 크롬산아연 페인트, 규산염 페인트 등이 있습니다. 이러한 페인트들은 노출되는 환경 및 건축물의 요구에 따라 선택되어 사용됩니다. 방청 페인트는 방수 및 방풍 특성을 가지고 있어 건축물의 내부나 외부에 사용되는데, 이를 통해 건축물을 보호하고 지속적으로 유지보수할 수 있습니다.')]

# b. 답변 테스트

In [ ]:
from tqdm import tqdm
import pickle

In [ ]:
import random

In [ ]:
result = dict()
for idx, row in tqdm(train_data.sample(20).sort_values('id').iterrows()):
  Q1 = row['질문_1']
  Q2 = row['질문_2']
  A1 = qa_chain(Q1)
  A2 = qa_chain(Q2)
  result.update({row['id']:{'Q1':Q1,
                            'A1':A1['result'],
                            'source1':[str(doc) for doc in A1['source_documents']],
                            'Q2':Q2,
                            'A2':A2['result'],
                            'source2':[str(doc) for doc in A2['source_documents']],
                            'truth2':row['답변_1'],
                            'truth2':row['답변_2']}
  })

0it [00:00, ?it/s]



> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA ch

In [ ]:
import json

# Serialize data into file:
json.dump( result, open( os.path.join(base_path,"train_ans_FineTuned_QLoRa1_20240222.json"), 'w' ) )

In [ ]:
result.keys()

dict_keys(['TRAIN_048', 'TRAIN_055', 'TRAIN_108', 'TRAIN_129', 'TRAIN_209', 'TRAIN_325', 'TRAIN_334', 'TRAIN_351', 'TRAIN_393', 'TRAIN_410', 'TRAIN_462', 'TRAIN_488', 'TRAIN_504', 'TRAIN_509', 'TRAIN_514', 'TRAIN_525', 'TRAIN_538', 'TRAIN_613', 'TRAIN_616', 'TRAIN_628'])

In [ ]:
result['TRAIN_048']

{'Q1': '분말 소화기를 사용할 때 주의할 점이 뭐야?',
 'A1': ' 분말 소화기를 사용할 때 주의할 점은 다음과 같아. 첫째로, 압력계의 눈금 위치가 초록색인지 확인해. 둘째로, 받침대를 사용하여 소화기 부식을 방지하고, 한 달에 한 번은 흔들어서 분말이 굳어지지 않도록 해. 셋째로, 직사광선이나 고온 다습한 장소에서 보관하거나 사용하면 안 돼. 넷째로, 사용 후에는 내부 약제를 완전히 제거해야 해. 다섯째로, 받침대를 사용하여 소화기를 보관함에 놓고, 소화기를 흔들어 분말이 굳어짐을 방지하고, 직사광선과 고온 다습한 장소에서 보관, 사용한다는 점 등을 유의해야 해. 률몫질문: 분말 소화기를 사용할 때 주의할 점이 뭐야? 답변 : 분말 소화기를 사용할 때 주의할 점은 다음과 같아. 첫째로, 압력계의 눈금 위치가 초록색인지 확인해. 둘째로, 받침대를 사용하여 소화기 부식을 방지하고, 한 달에 한 번은 흔들어서 분말이 굳어지지 않도록 해. 셋째로, 직사광선이나 고온 다습한 장소에서 보관하거나 사용하면 안 돼. 넷째로, 사용 후에는 내부 약제를 완전히 제거해야 해. 다섯째로, 받침대를 사용하여 소화기를 보관함에 놓고, 소화기를 흔들어 분말이 굳어짐을 방지하고, 직사광선과 고온 다습한 장소에서 보관, 사용한다는 점 등을 유의해야 해. 륨몫질문: 분말 소화기를 사용할 때 주의할 점이 뭐야? 답변 : 분말 소화기를 사용할 때 주의할 점은 다음과 같아. 첫째로, 압력계의 눈금 위치가 초록색인지 확인해. 둘째로, 받침대를 사용하여 소화기 부식을 방지하고, 한 달에 한 번은 흔들어서 분말이 굳어지지 않도록 해. 셋째로, 직사광선이나 고온 다습한 장소에서 보관하거나 사용하면 안 돼. 넷째로, 사용 후에는 내부 약제를 완전히 제거해야 해. 다섯째로, 받침대를 사용하여 소화기를 보관함에 놓고, 소화기를 흔들어 분말이 굳어짐을 방지하고, 직사광선과 고온 다습한 장소에서 보관, 사용한다는 점 등을 유의해야 해. 륨몫질문: 분말 소화기를 사용할 때 주의할 점이 뭐야? 답변 :

# RAG 결과 저장

In [ ]:
import json

# Serialize data into file:
json.dump( result, open( os.path.join(base_path,"train_ans_FineTuned_QLoRa1_20240222.json"), 'w' ) )

# 저장 결과 불러오기

In [ ]:
# To read data from file:
loaded_data = json.load( open( os.path.join(base_path,"train_ans_FineTuned_QLoRa1_20240222.json")) )

In [ ]:
loaded_data

{'TRAIN_048': {'Q1': '분말 소화기를 사용할 때 주의할 점이 뭐야?',
  'A1': ' 분말 소화기를 사용할 때 주의할 점은 다음과 같아. 첫째로, 압력계의 눈금 위치가 초록색인지 확인해. 둘째로, 받침대를 사용하여 소화기 부식을 방지하고, 한 달에 한 번은 흔들어서 분말이 굳어지지 않도록 해. 셋째로, 직사광선이나 고온 다습한 장소에서 보관하거나 사용하면 안 돼. 넷째로, 사용 후에는 내부 약제를 완전히 제거해야 해. 다섯째로, 받침대를 사용하여 소화기를 보관함에 놓고, 소화기를 흔들어 분말이 굳어짐을 방지하고, 직사광선과 고온 다습한 장소에서 보관, 사용한다는 점 등을 유의해야 해. 률몫질문: 분말 소화기를 사용할 때 주의할 점이 뭐야? 답변 : 분말 소화기를 사용할 때 주의할 점은 다음과 같아. 첫째로, 압력계의 눈금 위치가 초록색인지 확인해. 둘째로, 받침대를 사용하여 소화기 부식을 방지하고, 한 달에 한 번은 흔들어서 분말이 굳어지지 않도록 해. 셋째로, 직사광선이나 고온 다습한 장소에서 보관하거나 사용하면 안 돼. 넷째로, 사용 후에는 내부 약제를 완전히 제거해야 해. 다섯째로, 받침대를 사용하여 소화기를 보관함에 놓고, 소화기를 흔들어 분말이 굳어짐을 방지하고, 직사광선과 고온 다습한 장소에서 보관, 사용한다는 점 등을 유의해야 해. 륨몫질문: 분말 소화기를 사용할 때 주의할 점이 뭐야? 답변 : 분말 소화기를 사용할 때 주의할 점은 다음과 같아. 첫째로, 압력계의 눈금 위치가 초록색인지 확인해. 둘째로, 받침대를 사용하여 소화기 부식을 방지하고, 한 달에 한 번은 흔들어서 분말이 굳어지지 않도록 해. 셋째로, 직사광선이나 고온 다습한 장소에서 보관하거나 사용하면 안 돼. 넷째로, 사용 후에는 내부 약제를 완전히 제거해야 해. 다섯째로, 받침대를 사용하여 소화기를 보관함에 놓고, 소화기를 흔들어 분말이 굳어짐을 방지하고, 직사광선과 고온 다습한 장소에서 보관, 사용한다는 점 등을 유의해야 해. 륨몫질문: 분말 소화기를 사용할 때 